# Amazon Bedrock AgentCore Memory with AgentCore SDK Session Management

## Introduction

This notebook demonstrates how to add memory (short-term and long-term) to Strands Agents using `AgentCore Memory Session Manager`, a session management package offered in AgentCore SDK. 

What you will learn:

* Create a shared memory resource
* Configure long-term memory strategy and manage data logical isolation with namespaces
* Congiture Strands Agents to use AgentCore Memory Session Manager to store and retrievel short and long term memories


## Scenario context

In the previous session, you have taken the mortgage assistant built with Strands to AgentCore Runtime, added tools on AgentCore Gateway, authentication and authorisation with AgentCore Identity. In this session, we continue the journey to make the agent more helpful by adding a memory component using AgentCore Memory.

## Architecture

![agentcore_memory_architecture.png](images/option-2-architecture-agentcore-memory-strands-session-manager.png)

1. Whenever a conversation is added, whether it is from the user or agent, the conversation will be stored in AgentCore Memory as short-term memory by Strands Agents AgentCore Memory Session Manager. 
2. Whenever an agent starts, AgentCore Memory Session Manager loads the recent conversations from short-term memory.
3. A memory extraction module works asynchronously in the background, and use the defined strategy to turn short-term memory into long-term memory.
4. Whenever a user asks a question, AgentCore Memory Session Manager runs a natural language query against long-term memory and obtain related information as context. 

For details on how Strands Agents interact with AgentCore Memory, check the section `Understanding how AgentCore Memory Session Manager works`.

## Step 0: Environment Setup

In [ ]:
import logging
import boto3
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## Step 1: Create AgentCore Memory Resource
First, create AgentCore Memory resource for the mortgage assistant in your AWS account.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "mortgage_assistant_memory"
memory_id = None

In [ ]:
# for testing only: if agentcore memory has already been created outside this workshop
# memory_id="mortgage_assistant_20250817122625-BgMl7M34lQ"

In the following code, we create the AgentCore Memory resource.

Notice we create `strategies` here and use the them when creating AgentCore Memory. When `strategies` are provided, AgentCore Memory will use the provided strategies to process short-term memories (conversation histories) and generate long-term memories, such as user facts, user preference or conversation summaries.

If AgentCore Memory is created without `strategies`, it only keeps short-term memories (conversation histories). No long-term memories will be generated.

The AgentCore Memory resource will take a few minutes to create. 

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")

    # Strategies for long term memory
    strategies = [
        {
            StrategyType.SEMANTIC.value: {
                "name": "fact_extractor",
                "description": "Extracts and stores factual information",
                "namespaces": ["mortgage_assistant/{actorId}/facts"],
            },
        },
        {
            StrategyType.SUMMARY.value: {
                "name": "conversation_summary",
                "description": "Captures summaries of conversations",
                "namespaces": ["mortgage_assistant/{actorId}/{sessionId}"],
            },
        },
        {
            StrategyType.USER_PREFERENCE.value: {
                "name": "user_preferences",
                "description": "Captures user preferences and settings",
                "namespaces": ["mortgage_assistant/{actorId}/preferences"],
            },
        },
    ]

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Mortgage Assistant Memory", # Human-readable description
        strategies=strategies,                  # Strategies is for long term memory. For short term memory only, pass in an empty array 
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

Now the AgentCore memory is created. You can go to your AWS Console -> Amazon Bedrock AgentCore -> Memory to see the resource. 

Please note, `memory_id` will be used in all API calls to store/retrieve memory. We will store the memory ID in System Manager Parameter Store, so our AgentCore Runtime can fetch this memory ID at runtime to access the AgentCore Memory we created.

In [ ]:
ssm = boto3.client("ssm")
param_name = "/app/mortgage_assistant/agentcore/memory_id"

def store_memory_id_in_ssm(param_name: str, memory_id: str):
    ssm.put_parameter(Name=param_name, Value=memory_id, Type="String", Overwrite=True)
    print(f"Stored memory_id in SSM: {param_name}")
    
store_memory_id_in_ssm(param_name, memory_id)

## Step 3: Create Multi-Agent Architecture with Strands Agents
Now we can re-create the AgentCore Runtime for the Strands agents as you have done in the last few labs. The execution role for AgentCore Runtime needs access to the AgentCore Memory, and the System Manager Parameter Store to access the Memory ID. This is handled for you in this lab through a project-wide util function.

In [ ]:
from utils import create_agentcore_role

agent_name="mortgage_assistant"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

Now we can configure the AgentCore Runtime. We set `entrypoint` as `option_2_memory_strands_session_manager.py`. Open up this Python file to see how AgentCore Memory is used by Strands Agents.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

agentcore_runtime = Runtime()
response = agentcore_runtime.configure(
    entrypoint="option_2_memory_strands_session_manager.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Now we will launch the AgentCore Runtime resource in your AWS account. It will take a few minutes as the docker image is built in Code Build and deployed to AgentCore Runtime.

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

Now you can see the AgentCore Runtime resource in your AWS account in Amazon Bedrock AgentCore -> Agent Runtime. But it won't be ready until the status is changed to `READY`

As usual, we check the status of AgentCore runtime until it is ready.

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Once the status is shown as `READY`, the AgentCore Runtime is ready to be invoked.

## Step 3: Invoke the deployed agent

First, let's create a `user_id` and `session_id` for testing. We also create `user_id_for_testing_no_memory` and `session_id_for_testing_no_memory` to test memory isolation. 

In [ ]:
import uuid

# An randomly generated fictional user ID. 
# In a real system, replace this with a real user ID in your system
user_id=str(uuid.uuid4())
user_id_for_testing_no_memory=str(uuid.uuid4())

session_id = str(uuid.uuid4())
session_id_for_testing_no_memory = str(uuid.uuid4())

print(f"user_id: {user_id}")
print(f"user_id_for_testing_no_memory: {user_id_for_testing_no_memory}")
print(f"session_id: {session_id}")
print(f"session_id_for_testing_no_memory: {session_id_for_testing_no_memory}")

We create a convenient method for printing the reponse messages nicely.

In [ ]:
def print_response_text(invoke_response):
    response_bytes=invoke_response['response']
    response_strings = [item.decode('utf-8') for item in response_bytes]
    # print(response_bytes)
    response_combined_string = "".join(response_strings)
    # print(response_combined_string)
    
    response_dict = json.loads(response_combined_string)
    response_text=response_dict["result"]["content"][0]["text"]
    print(response_text)

In [ ]:
sample_prompt_1 = "Hey I am John. I'd like to check my existing home loan. My customer id is ABC-123"
sample_prompt_2 = "We are planning to upsize. We are shopping around to see what is avaialble?"
sample_prompt_3 = "Do do banks decide how much it can lend me?"
sample_prompt_4 = "What is the difference between standard variable and fixed rate"
sample_prompt_5 = "What is interest only vs principal and interest?"

Now we call the agent a few times. 

In [ ]:
import pprint
import json

invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_1,
    "user_id": user_id
}, session_id=session_id)
# pprint.pprint(invoke_response, width=80, depth=None)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_2,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_3,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_4,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

## Step 4: Check memory

After calling the agents a few times, we can now check the memory. We use two ways to check the memory.

1. We will check the memory with the user_id and session_id with Bedrock API. This allows us to check the stored short-term and long-term memory.
2. We will check the CloudWatch log to see the exact memory operations at the lifecycle events in supervisor and sub-agents. This helps us to adapt our memory management to suit our use case.

### 4.1 Check short-term memory
First, we create a convenient method for listing out the last K conversation turns from the short-term memory.

In [ ]:
# Check what's stored in memory
def list_last_k_turns(memory_id, actor_id, session_id, k):
    print("=== Memory Contents ===")
    print(f"actor_id: {actor_id}")
    print(f"session_id: {session_id}")
    
    recent_turns = client.get_last_k_turns(
        memory_id=memory_id,
        actor_id=actor_id,
        session_id=session_id,
        k=k # Adjust k to see more or fewer turns
        # branch_name="main"
    )
    
    for i, turn in enumerate(recent_turns, 1):
        print(f"Turn {i}:")
        for message in turn:
            role = message['role']
            content = json.loads(message['content']['text'])
            # content = message['content']['text'][:400] + " <🖨️ ... omitted for printing only ... 🖨️>" if len(message['content']['text']) > 400 else message['content']['text']
            print(f"  {role}: {json.dumps(content, indent=4)}")
        print()

Using a valid `user_id` and `session_id`, we expect to see the recent conversation history.

In [ ]:
list_last_k_turns(memory_id, f"{user_id}", session_id, 20)

Next, check the memory with a different user_id `user_id_for_testing_no_memory` which does not exist. We expect to see nothing.

In [ ]:
list_last_k_turns(memory_id, f"{user_id_for_testing_no_memory}-supervisor", session_id, 10)

Check the memory with the same `user_id`, but a different session_id `session_id_for_testing_no_memory`. Again, we expect to see NO messages.

In [ ]:
list_last_k_turns(memory_id, f"{user_id}", session_id_for_testing_no_memory, 10)

#### CloudWatch Log
Let's find the the log file in CloudWatch. You should see the short-term memory loaded when the agent is invoked. As we rely on AgentCore Memory Session Manager's default behaviour, we do not see the full message in the log.

![option-2-short-term-memory-retrieval.png](images/option-2-short-term-memory-retrieval.png)

You should also see memory created event as the current conversation is stored to memory.

![option-2-create-memory-event.png](images/option-2-create-memory-event.png)

### 4.2 Check long-term memory
Again, we will first use `list_memory_records` method to see the long-term memory.

In [ ]:
agentcore_client = boto3.client("bedrock-agentcore")

print("----------")
print("Long term memory - facts:")
response_facts = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/facts",
    maxResults=10
)
# pprint.pprint(response_facts, width=80, depth=None)
for list_memory_record in response_facts["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - preferences:")
response_preferences = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/preferences",
    maxResults=10
)
# pprint.pprint(response_preferences, width=80, depth=None)
for list_memory_record in response_preferences["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - summary:")
response_summary = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/{session_id}",
    maxResults=10
)
# pprint.pprint(response_summary, width=80, depth=None)
for list_memory_record in response_summary["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

Alternatively, we can also use `retrieve_memories` method to retrieve long-term memory.

In [ ]:
memory_client=MemoryClient(region_name=region)
print("----------")
print("Long term memory - facts:")
facts_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/facts", query="all", top_k=10
        )
# to see the raw facts memory records
# print(facts_memory)
# print the content
for memory in  facts_memory:
    print(memory["content"]["text"])

print("----------")
print("Long term memory - preferences:")
prefs_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/preferences", query="all", top_k=10
        )
# print the content
for memory in  prefs_memory:
    memory_content = memory["content"]["text"]
    memory_content_json = json.loads(memory_content)
    print(json.dumps(memory_content_json, indent=4))

print("----------")
print("Long term memory - summaries:")
summaries_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/{session_id}", query="all", top_k=10
        )
# print the content
for memory in  summaries_memory:
    print(memory["content"]["text"])
print("----------")

#### CloudWatch
Let's find the the log file in CloudWatch. You should see long term memory loaded from the namespaces we defined.

![option-2-long-term-memory-retrieval.png](images/option-2-long-term-memory-retrieval.png)

## Understanding how AgentCore Memory Session Manager works

The beauty of the AgentCore Memory Session Manager approach is that it's automatic and transparent - developers don't need to manually manage memory operations. The session manager handles everything through Strands' lifecycle hooks. On the flip side, it might be hard at the start to reason how everything works under the hood. We will uncover how AgentCore Memory Session Manager works in this section.

Source code of AgentCore Memory Session Manager: https://github.com/aws/bedrock-agentcore-sdk-python/blob/main/src/bedrock_agentcore/memory/integrations/strands/session_manager.py



#### 1. When Messages Are Written to Memory

  Event: `MessageAddedEvent` (after agent or user creates a message)

  The `append_message()` method is called by the Strands framework when a message is added to the conversation. This happens:
  - After a user sends a message to the agent
  - After the agent generates a response

  How it works:

<pre>
  def append_message(self, message: Message, agent: "Agent", **kwargs: Any) -> None:
      created_message = self.create_message(
          self.session_id,
          agent.agent_id,
          SessionMessage.from_message(message, 0)
      )
</pre>

  The create_message() method then calls `memory_client.create_event()` to persist the message to AgentCore Memory with a monotonically increasing timestamp.

  ---
  #### 2. When Short-Term Memory is Read

  Event: Agent Initialization (when the agent first starts or a conversation begins)

  The `initialize()` method is called by Strands when setting up the agent session. This happens:
  - At the start of a conversation
  - When creating a new agent instance for a session

  How it provides short-term memory:
  The session manager inherits from `RepositorySessionManager`, which automatically:
  1. Loads previous messages from the session using `list_messages()`
  2. Populates the agent's conversation history before processing begins
  3. Makes recent conversation turns available as context

  The parent class handles the actual loading of short-term memory into the agent's message list, so the agent has access to recent conversation history when it starts processing.

  ---
  #### 3. When Long-Term Memory is Retrieved

  Event: `MessageAddedEvent` (specifically for user messages)

  The `retrieve_customer_context()` method is triggered via a registered hook:

<pre>
  def register_hooks(self, registry: HookRegistry, **kwargs) -> None:
      RepositorySessionManager.register_hooks(self, registry, **kwargs)
      registry.add_callback(
          MessageAddedEvent,
          lambda event: self.retrieve_customer_context(event)
      )
</pre>

  How it provides long-term memory:
  1. Triggered: Every time a message is added (typically user messages)
  2. Extracts query: Gets the user's question from the message
  3. Retrieves memories: Calls memory_client.retrieve_memories() across configured namespaces (facts, preferences, etc.)
  4. Injects context: Adds retrieved memories as a synthetic assistant message with <user_context> tags before the user's message

  This means long-term memory context is automatically injected into the conversation flow, appearing to the agent as additional context about the user.
  
  ---


#### Detailed Timeline

<pre>
  ┌─────────────────────────────────────────────────────────────────────┐
  │ AGENT INITIALIZATION                                                │
  └─────────────────────────────────────────────────────────────────────┘
      │
      ├─► initialize() called
      │   └─► Loads short-term memory from AgentCore Memory
      │       └─► Retrieves last N conversation turns
      │           └─► Populates agent.messages with history
      │
      ├─► Agent system prompt includes recent conversation context
      │
      ├─► Hooks registered:
      │   ├─► MessageAddedEvent → append_message() (from parent)
      │   └─► MessageAddedEvent → retrieve_customer_context()
      │
      └─► Agent ready and waiting


  ┌─────────────────────────────────────────────────────────────────────┐
  │ USER SENDS MESSAGE: "What mortgage options do I have?"              │
  └─────────────────────────────────────────────────────────────────────┘
      │
      ├─► User message added to agent.messages
      │
      ├─► MessageAddedEvent FIRED
      │   │
      │   ├─► [Hook 1] append_message() triggered
      │   │   └─► create_message() called
      │   │       └─► memory_client.create_event()
      │   │           └─► Stores: "What mortgage options do I have?"
      │   │               (original message only, no context yet)
      │   │
      │   └─► [Hook 2] retrieve_customer_context() triggered
      │       ├─► Extracts query: "What mortgage options do I have?"
      │       ├─► memory_client.retrieve_memories()
      │       │   ├─► Namespace: mortgage_assistant/{userId}/facts
      │       │   └─► Namespace: mortgage_assistant/{userId}/preferences
      │       │
      │       ├─► Retrieved:
      │       │   - "John prefers fixed rate mortgages"
      │       │   - "Income: $150k/year"
      │       │   - "Planning to upsize"
      │       │
      │       └─► Injects synthetic message into agent.messages:
      │           {
      │             "role": "assistant",
      │             "content": [{
      │               "text": "<user_context>
      │                 John prefers fixed rate mortgages
      │                 Income: $150k/year
      │                 Planning to upsize
      │                 </user_context>"
      │             }]
      │           }
      │
      ├─► Current agent.messages now contains:
      │   [
      │     { role: "user", content: "..." },           // from STM
      │     { role: "assistant", content: "..." },      // from STM
      │     { role: "assistant", content: "<user_context>..." }, // LTM (injected)
      │     { role: "user", content: "What mortgage options..." } // current
      │   ]
      │
      └─► BeforeModelCallEvent FIRED
          └─► [Hook if logging enabled] Logs complete message payload
              └─► This is what you'd see with BeforeModelCallEvent hook


  ┌─────────────────────────────────────────────────────────────────────┐
  │ LLM PROCESSES REQUEST                                               │
  └─────────────────────────────────────────────────────────────────────┘
      │
      ├─► LLM receives:
      │   - System prompt
      │   - Short-term memory (recent conversation)
      │   - Long-term memory context (<user_context> tags)
      │   - Current user question
      │
      └─► Generates personalized response


  ┌─────────────────────────────────────────────────────────────────────┐
  │ AGENT GENERATES RESPONSE                                            │
  └─────────────────────────────────────────────────────────────────────┘
      │
      ├─► Agent response added to agent.messages
      │   "Based on your preference for fixed rates and $150k income,
      │    I recommend our 30-year fixed rate mortgage at 6.5%..."
      │
      ├─► MessageAddedEvent FIRED (again)
      │   │
      │   ├─► [Hook 1] append_message() triggered
      │   │   └─► create_message() called
      │   │       └─► memory_client.create_event()
      │   │           └─► Stores: "Based on your preference..."
      │   │               (agent response only, NOT the <user_context>)
      │   │
      │   └─► [Hook 2] retrieve_customer_context() triggered
      │       └─► Skipped (only processes user messages)
      │
      └─► Response returned to user


  ┌─────────────────────────────────────────────────────────────────────┐
  │ WHAT'S STORED IN AGENTCORE MEMORY                                   │
  └─────────────────────────────────────────────────────────────────────┘

  SHORT-TERM MEMORY (Event Store):
    ┌─────────────────────────────────────────────────────────────┐
    │ Turn 1:                                                     │
    │   User: "What mortgage options do I have?"                  │
    │   Agent: "Based on your preference for fixed rates..."      │
    │                                                             │
    │ Turn 2:                                                     │
    │   User: "What about interest-only?"                         │
    │   Agent: "Interest-only mortgages are available..."         │
    └─────────────────────────────────────────────────────────────┘

  LONG-TERM MEMORY (Extracted asynchronously):
    ┌─────────────────────────────────────────────────────────────┐
    │ Namespace: mortgage_assistant/{userId}/facts                │
    │   - John prefers fixed rate mortgages                       │
    │   - Income: $150k/year                                      │
    │   - Planning to upsize                                      │
    │                                                             │
    │ Namespace: mortgage_assistant/{userId}/preferences          │
    │   - Prefers 30-year terms                                   │
    │   - Risk-averse borrower                                    │
    └─────────────────────────────────────────────────────────────┘

</pre>

## Summary 
You have added memory to your multi-agent Mortgage Assistant. 

You can now move to the next session.